In [0]:
from pyspark.sql.functions import to_date, first, col, round

In [0]:
df_taxa = spark.read.parquet("dbfs:/databricks-results/bronze/*/*/*")


In [0]:
moedas = ["USD", "EUR", "GBP"]

df_moedas = df_taxa.filter(df_taxa.moeda.isin(moedas))

+-----+--------+----------+
|moeda|    taxa|      data|
+-----+--------+----------+
|  USD|0.183143|2024-07-05|
|  USD|0.182634|2024-07-06|
|  USD|0.182003|2024-07-07|
|  USD|0.182752|2024-07-08|
|  USD| 0.18455|2024-07-09|
|  USD|0.184631|2024-07-10|
|  USD| 0.18384|2024-07-11|
|  USD|0.184186|2024-07-13|
|  USD|0.184355|2024-07-14|
|  USD|0.183543|2024-07-15|
|  USD|0.182176|2024-07-17|
|  USD|0.180352|2024-07-18|
|  USD|0.178446|2024-07-19|
|  USD|0.178568|2024-07-20|
|  USD| 0.18038|2024-07-21|
|  USD|0.179462|2024-07-22|
|  USD|0.176831|2024-07-24|
|  USD| 0.17716|2024-07-25|
|  USD|0.177193|2024-07-27|
|  USD|0.178171|2024-07-30|
+-----+--------+----------+
only showing top 20 rows



In [0]:
df_moedas = df_moedas.withColumn("data", to_date("data"))

In [0]:
df_moedas_ordenado = (df_moedas.groupBy("data")
                                .pivot("moeda")
                                .agg(first("taxa"))
                                .orderBy("data", ascending=False))

+----------+--------+--------+--------+
|      data|     EUR|     GBP|     USD|
+----------+--------+--------+--------+
|2024-08-23|0.160773|0.136222|0.178808|
|2024-08-22|0.161645|0.137244|0.179636|
|2024-08-21|0.163487|0.139287|0.182349|
|2024-08-20|0.164002|0.140013|0.182495|
|2024-08-19|0.166835|0.142378|0.184942|
|2024-08-18|0.165744|0.141152|0.182722|
|2024-08-17|0.165592|0.141169|0.182722|
|2024-08-16|0.165592|0.141076|0.182722|
|2024-08-15|0.166101|0.141741|0.182269|
|2024-08-13|0.166639|0.142424|0.183244|
|2024-08-12|0.166457|0.142546|0.182037|
|2024-08-11|0.166365|0.142405|0.181561|
|2024-08-10|0.165137|0.141383|0.180369|
|2024-08-09|0.166281| 0.14267|0.181619|
|2024-08-08|0.165141|0.141413|0.180258|
|2024-08-07|0.162329|0.139915| 0.17739|
|2024-08-06| 0.16182|0.139393|0.176772|
|2024-08-05| 0.15958|0.136646|0.174746|
|2024-08-04|0.160008|0.136521|0.174575|
|2024-08-03|0.159859|0.136317|0.174575|
+----------+--------+--------+--------+
only showing top 20 rows



In [0]:
df_moedas_real = df_moedas_ordenado.select("*")

+----------+--------+--------+--------+
|      data|     EUR|     GBP|     USD|
+----------+--------+--------+--------+
|2024-08-23|0.160773|0.136222|0.178808|
|2024-08-22|0.161645|0.137244|0.179636|
|2024-08-21|0.163487|0.139287|0.182349|
|2024-08-20|0.164002|0.140013|0.182495|
|2024-08-19|0.166835|0.142378|0.184942|
|2024-08-18|0.165744|0.141152|0.182722|
|2024-08-17|0.165592|0.141169|0.182722|
|2024-08-16|0.165592|0.141076|0.182722|
|2024-08-15|0.166101|0.141741|0.182269|
|2024-08-13|0.166639|0.142424|0.183244|
|2024-08-12|0.166457|0.142546|0.182037|
|2024-08-11|0.166365|0.142405|0.181561|
|2024-08-10|0.165137|0.141383|0.180369|
|2024-08-09|0.166281| 0.14267|0.181619|
|2024-08-08|0.165141|0.141413|0.180258|
|2024-08-07|0.162329|0.139915| 0.17739|
|2024-08-06| 0.16182|0.139393|0.176772|
|2024-08-05| 0.15958|0.136646|0.174746|
|2024-08-04|0.160008|0.136521|0.174575|
|2024-08-03|0.159859|0.136317|0.174575|
+----------+--------+--------+--------+
only showing top 20 rows



In [0]:
for moeda in moedas:
    df_moedas_real = df_moedas_real.withColumn(moeda, round(1/col(moeda),4))


In [0]:
df_moedas_ordenado = df_moedas_ordenado.coalesce(1)
df_moedas_real = df_moedas_real.coalesce(1)

In [0]:
(df_moedas_ordenado.write
                    .mode("overwrite")
                    .format("csv")
                    .option("header", "true")
                    .save("dbfs:/databricks-results/prata/taxa_conversao"))

(df_moedas_real.write
                    .mode("overwrite")
                    .format("csv")
                    .option("header", "true")
                    .save("dbfs:/databricks-results/prata/valores_reais"))